# NOTA: Tratamiento de valores atípicos en la base *Italia*

### Identificación de *outliers*
 - **Método:** Visualización con diagramas de caja (*boxplots*) por cada columna numérica.
 - **Justificación:** Los *boxplots* muestran la distribución y señalan valores alejados del rango intercuartílico (IQR),
   facilitando la detección visual de *outliers*.

 ---

 ### Exclusión de anfitriones atípicos
 - **Método:** Eliminación de registros con `id_anfitrion` en {2634978, 579781108, 321429178}.
 - **Justificación:** Estos anfitriones exhiben patrones extremos (p. ej., muchos listados o `noches_minimas = 999`),
   lo que sesga el análisis y no representa el comportamiento típico del mercado.

 ---

### Verificación de calificaciones
 - **Método:** Validación de que las variables de calificación (p. ej., `calif_total`, `calif_exactitud`, etc.) estén en el rango [0, 5].
 - **Justificación:** Las calificaciones están acotadas de forma natural, la verificación asegura integridad y evita registros mal capturados.

 ---

### Exportación de registros excluidos
 - **Método:** Exportación de los registros filtrados a `anfitriones_outliers.csv`.
 - **Justificación:** Conserva los datos atípicos para análisis posterior o auditoría, sin afectar el dataset principal.


In [234]:
# =========================
# Importar librerías
# =========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [235]:
# ===========================================================
# CARGA Y PREPARACIÓN
# ============================================================
df = pd.read_csv("italia_filtrado_es_sin_NAN.csv")


In [236]:
# =========================================================
# IDs de los anfitriones a excluir
# =========================================================
anfitriones_excluidos = [2634978, 579781108, 321429178]

In [237]:
# =========================================================
# Identificar y manejar los valores sentinela
# =========================================================
sentinel_dict = {
    'resenas_por_mes': -222,
    'calif_total': -222,
    'calif_exactitud': -222,
    'calif_limpieza': -222,
    'calif_checkin': -222,
    'calif_comunicacion': -222,
    'calif_ubicacion': -222,
    'calif_valor': -222,
    'tasa_respuesta_host': 222,
    'tasa_aceptacion_host': 222
}


In [238]:
# ===========================================================
# EXPLORACIÓN VISUAL DE OUTLIERS
# ===========================================================
# Crear una copia del DataFrame para esta sección
df_temp = df.copy()

# Manejar valores sentinela para la visualización
for col, sentinel in sentinel_dict.items():
    if col in df_temp.columns:
        df_temp[col] = df_temp[col].replace(sentinel, np.nan)

# Obtener las columnas numéricas
numeric_cols = df_temp.select_dtypes(include=np.number).columns.tolist()

# Crear la carpeta para guardar los gráficos si no existe
folder_name = 'graficas_outliers'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Carpeta '{folder_name}' creada ✅")

# Generar y guardar un box plot individual para cada columna numérica
plt.style.use('fivethirtyeight')
for col in numeric_cols:
    plt.figure(figsize=(10, 2))
    df_temp[col].plot(kind='box', vert=False)
    plt.title(f'Diagrama de caja de {col}', fontsize=12)
    plt.tight_layout()
    plt.savefig(f'{folder_name}/boxplot_{col}.png')
    plt.close()

print(f"Diagramas de caja individuales guardados en la carpeta '{folder_name}' ✅")

Diagramas de caja individuales guardados en la carpeta 'graficas_outliers' ✅


In [239]:
# ===========================================================
# VERIFICACIÓN DE RANGOS DE CALIFICACIÓN (0-5)
# ===========================================================
calif_cols = [
    'calif_total', 'calif_exactitud', 'calif_limpieza', 'calif_checkin',
    'calif_comunicacion', 'calif_ubicacion', 'calif_valor'
]

print("\nVerificando que los valores de calificación estén entre 0 y 5:")
for col in calif_cols:
    # Excluir los valores NaN para la verificación
    valid_values = df_temp[col].dropna()
    is_in_range = valid_values.between(0, 5).all()
    if is_in_range:
        print(f"  - '{col}': Todos los valores están en el rango de 0 a 5. ✅")
    else:
        print(f"  - '{col}': ¡ATENCIÓN! Se encontraron valores fuera del rango de 0 a 5. ❌")


Verificando que los valores de calificación estén entre 0 y 5:
  - 'calif_total': Todos los valores están en el rango de 0 a 5. ✅
  - 'calif_exactitud': Todos los valores están en el rango de 0 a 5. ✅
  - 'calif_limpieza': Todos los valores están en el rango de 0 a 5. ✅
  - 'calif_checkin': Todos los valores están en el rango de 0 a 5. ✅
  - 'calif_comunicacion': Todos los valores están en el rango de 0 a 5. ✅
  - 'calif_ubicacion': Todos los valores están en el rango de 0 a 5. ✅
  - 'calif_valor': Todos los valores están en el rango de 0 a 5. ✅


In [240]:
# ===========================================================
# Paso 1: Calcular estadísticas antes de excluir a los anfitriones
# ===========================================================
df_antes = df.copy()
for col, sentinel in sentinel_dict.items():
    if col in df_antes.columns:
        df_antes[col] = df_antes[col].replace(sentinel, np.nan)

numeric_cols_antes = df_antes.select_dtypes(include=np.number).columns.tolist()
stats_antes = df_antes[numeric_cols_antes].agg(['min', 'mean', 'max'])


In [241]:
# ===========================================================
# Paso 2: Excluir a los anfitriones y calcular estadísticas
# ===========================================================
df_despues = df[~df['id_anfitrion'].isin(anfitriones_excluidos)].copy()
for col, sentinel in sentinel_dict.items():
    if col in df_despues.columns:
        df_despues[col] = df_despues[col].replace(sentinel, np.nan)

numeric_cols_despues = df_despues.select_dtypes(include=np.number).columns.tolist()
stats_despues = df_despues[numeric_cols_despues].agg(['min', 'mean', 'max'])


In [242]:
# ===========================================================
# Paso 3: Exportar ambos conjuntos de estadísticas a un solo archivo de Excel
# ===========================================================
with pd.ExcelWriter('estadisticas_comparativas_doble.xlsx') as writer:
    stats_antes.to_excel(writer, sheet_name='Estadisticas Antes de Excluir')
    stats_despues.to_excel(writer, sheet_name='Estadisticas Despues de Excluir')

In [243]:
# ===========================================================
# EXPORTAR REGISTROS DE ANFITRIONES EXCLUIDOS
# ===========================================================
# Filtrar el DataFrame para obtener solo los registros de los anfitriones excluidos
df_outliers = df[df['id_anfitrion'].isin(anfitriones_excluidos)]

# Exportar los registros filtrados a un nuevo archivo CSV
df_outliers.to_csv('anfitriones_outliers.csv', index=False)

print("Registros de los anfitriones outliers exportados a 'anfitriones_outliers.csv' ✅")


Registros de los anfitriones outliers exportados a 'anfitriones_outliers.csv' ✅


In [244]:
# ===========================================================
# EXPORTAR BASE DE DATOS SIN OUTLIERS
# ===========================================================
# Filtrar el DataFrame para excluir los registros de los anfitriones atípicos
df_sin_outliers = df[~df['id_anfitrion'].isin(anfitriones_excluidos)].copy()

# Exportar el DataFrame limpio a un nuevo archivo CSV
df_sin_outliers.to_csv('italia_sin_outliers.csv', index=False)

print("Base de datos sin outliers exportada a 'italia_sin_outliers.csv' ✅")

Base de datos sin outliers exportada a 'italia_sin_outliers.csv' ✅
